## **IMPORTS**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

## **FUNCIÓN DE SCRAPEADO**

In [3]:
def scrapeado_de_bebidas_lacteas(num_pagina):

  url = f"https://www.salemmaonline.com.py/lacteos/bebidas-lacteas?page={num_pagina}"
  headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", 
             "Accept-Encoding":"gzip, deflate", 
             "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", 
             "DNT":"1","Connection":"close", 
             "Upgrade-Insecure-Requests":"1"}
  
  r = requests.get(url, headers=headers)
  content = r.content
  soup = BeautifulSoup(content)
  

  alls = []
  for producto in soup.find_all('div', attrs={"class":"col-lg-3 col-md-4 col-sm-4 col-6 pdivp div-product-list"}):
      
      nombre_producto = producto.find('a', attrs={'class':'apsubtitle'})
      nombre_marca    = producto.find('h6', attrs={'class':'ptitle'})
      precio_producto = producto.find('h6', attrs={'class':'pprice'})

      all1=[]
      
      if nombre_producto is not None:
          all1.append(nombre_producto.text.strip())
      else:
          all1.append("Producto desconocido")

      if nombre_marca is not None:
          all1.append(nombre_marca.text.strip())
      elif nombre_marca is None:
          all1.append("Marca desconocida")

      if precio_producto is not None:
          all1.append(precio_producto.text.strip())
      else:
          all1.append('Precio desconocido')

      alls.append(all1)    
  return alls


## **LLAMADA A LA FUNCION DE SCRAPING**

In [10]:
resultados = []

for numero in range(1, 4):
  resultados.append(scrapeado_de_bebidas_lacteas(numero))

flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(resultados),columns=['Nombre del Producto','Marca','Precio'])
df.to_csv('bebidas_lacteas.csv', index=False, encoding='utf-8')

df

,Nombre del Producto,Marca,Precio
0,BEBIDA LACTEA X 1 LT,COOP,Gs. 3.800
1,BEBIDA LACTEA X 500 ML,COOP,Gs. 1.800
2,BEBIDA LACTEA LARGA VIDA X 500 ML,FAMLAC,Gs. 3.000
3,BEBIDA LACTEA ULTRA X 1 LT,LA PRADERA,Gs. 3.900
4,BEBIDA LACTEA LARGA VIDA X 1 LT,LOS COLONOS,Gs. 5.400
...,...,...,...
96,BEBIDA LACTEA VAINILLA X 140 ML,LA FORTUNA,Gs. 1.300
97,BEBIDA LACTEA VAINILLA X 350 ML,LA FORTUNA,Gs. 2.700
98,BEBIDA LACTEA VAINILLA X 500 ML,LA FORTUNA,Gs. 2.950
99,BEBIDA LACTEA VAINILLA X 1 LT,LA FORTUNA,Gs. 5.800


## **LIMPIEZA Y PREPARACION DE DATOS**

In [11]:
nombre_productos = []
presentaciones_productos = []
precios_productos = []
marcas_productos = []
es_uat = []
sabores_productos = []

posibles_sabores = ["VAINILLA", "BANANA", "UVA", "MANZANA", "FRUTILLA", "COCO", "DURAZNO"]
palabras_omitidas = ["BEBIDA", "LACTEA", "UAT", "UHT", "POTE", "SACHET"]

for producto in df["Nombre del Producto"]:
  
  #Separacion del nombre y la presentacion
  nombre_y_presentacion = producto.split(sep="X")

  presentaciones_productos.append(nombre_y_presentacion[1])

  lista_descripciones = nombre_y_presentacion[0].split(sep=" ")

  print(lista_descripciones)
  
  if ("UAT" in lista_descripciones) or ("UHT" in lista_descripciones):
    es_uat.append(True)
  else:
    es_uat.append(False)

  copy_lista_descripciones = lista_descripciones

  for elemento in copy_lista_descripciones:
    for palabra in palabras_omitidas:
      if str(palabra) == str(elemento):
        lista_descripciones.remove(elemento)
      else:
        continue
        

  print(lista_descripciones)

  for sabor in posibles_sabores:
    if sabor in lista_descripciones:
      sabores_productos.append(sabor)
      lista_descripciones.remove(sabor)
      break
  else:
      sabores_productos.append("N/A")

  print(lista_descripciones)

  nombre_productos = " ".join(lista_descripciones)




for producto in df["Precio"]:
  #Creacion del precio como numero entero
  moneda_y_precio = producto.split(sep=" ")
  precios_productos.append(int(moneda_y_precio[1].replace(".", "")))

for producto in df["Marca"]:
  #Correcion en el nombre de la marca CO-OP
  if producto == "CO OP":
    marcas_productos.append("CO-OP")
  elif producto == "COOP":
    marcas_productos.append("CO-OP")
  else:
    marcas_productos.append(producto)


df["Nombre del Producto"] = nombre_productos
df["Presentacion"] = presentaciones_productos
df["Precio_Int"] = precios_productos
df["Marca"] = marcas_productos
df["Sabor"] = sabores_productos
df["Es UAT"] = es_uat


df

['BEBIDA', 'LACTEA', '']
['LACTEA', '']
['LACTEA', '']
['BEBIDA', 'LACTEA', '']
['LACTEA', '']
['LACTEA', '']
['BEBIDA', 'LACTEA', 'LARGA', 'VIDA', '']
['LACTEA', 'LARGA', 'VIDA', '']
['LACTEA', 'LARGA', 'VIDA', '']
['BEBIDA', 'LACTEA', 'ULTRA', '']
['LACTEA', 'ULTRA', '']
['LACTEA', 'ULTRA', '']
['BEBIDA', 'LACTEA', 'LARGA', 'VIDA', '']
['LACTEA', 'LARGA', 'VIDA', '']
['LACTEA', 'LARGA', 'VIDA', '']
['BEBIDA', 'LACTEA', 'ULTRA', '']
['LACTEA', 'ULTRA', '']
['LACTEA', 'ULTRA', '']
['BEBIDA', 'LACTEA', 'FORTIFICADA', '']
['LACTEA', 'FORTIFICADA', '']
['LACTEA', 'FORTIFICADA', '']
['BEBIDA', 'LACTEA', 'LARGA', 'VIDA', '']
['LACTEA', 'LARGA', 'VIDA', '']
['LACTEA', 'LARGA', 'VIDA', '']
['BEBIDA', 'LACTEA', 'TROPICANA', 'BANANA', '']
['LACTEA', 'TROPICANA', 'BANANA', '']
['LACTEA', 'TROPICANA', '']
['BEBIDA', 'LACTEA', 'TROPICANA', 'VAINILLA', '']
['LACTEA', 'TROPICANA', 'VAINILLA', '']
['LACTEA', 'TROPICANA', '']
['BEBIDA', 'LACTEA', 'TROPICANA', 'BANANA', '']
['LACTEA', 'TROPICANA', 'BAN

,Nombre del Producto,Marca,Precio,Presentacion,Precio_Int,Sabor,Es UAT
0,LACTEA,CO-OP,Gs. 3.800,1 LT,3800,N/A,False
1,LACTEA,CO-OP,Gs. 1.800,500 ML,1800,N/A,False
2,LACTEA,FAMLAC,Gs. 3.000,500 ML,3000,N/A,False
3,LACTEA,LA PRADERA,Gs. 3.900,1 LT,3900,N/A,False
4,LACTEA,LOS COLONOS,Gs. 5.400,1 LT,5400,N/A,False
...,...,...,...,...,...,...,...
96,LACTEA,LA FORTUNA,Gs. 1.300,140 ML,1300,VAINILLA,False
97,LACTEA,LA FORTUNA,Gs. 2.700,350 ML,2700,VAINILLA,False
98,LACTEA,LA FORTUNA,Gs. 2.950,500 ML,2950,VAINILLA,False
99,LACTEA,LA FORTUNA,Gs. 5.800,1 LT,5800,VAINILLA,False
